# Project 4: 

In [1]:
# !pip install autotime
# !pip install nb-black
# !pip install cvxpy

In [2]:
%load_ext autotime
%load_ext nb_black

# General
import os
import time
import numpy as np
import cvxpy as cp
import scipy as sp
import pandas as pd
import itertools
from datetime import datetime

# Ignore warnings LOL
import warnings

warnings.simplefilter("ignore")

<IPython.core.display.Javascript object>

In [5]:
A = None

for folder in sorted(list(os.walk("./data/project 4 dataset/prediction/"))[0][1]):

    # Initial capacity
    init_data = pd.read_csv("./data/project 4 dataset/init_data.csv", index_col=[0])

    # Train data
    root, dirs, files = next(os.walk(f"./data/project 4 dataset/prediction/{folder}"))
    data = {}
    for file in files:
        if "csv" in file:
            try:
                train = pd.read_csv(os.path.join(root, file), index_col=[0]).iloc[:7]
                train = train.reindex(
                    [
                        datetime.strftime(
                            datetime.strptime(date, "%Y/%m/%d"), "%Y-%m-%d"
                        )
                        if "/" in date
                        else date
                        for date in train.index
                    ]
                )
                data[os.path.basename(file).replace(".csv", "")] = train
            except:
                import pdb

                pdb.set_trace()
    # Test data
    root, dirs, files = next(os.walk("./data/project 4 dataset/eval"))
    for file in files:
        if "csv" in file:
            try:
                test = pd.read_csv(
                    os.path.join(root, file),
                    header=None,
                    index_col=[0],
                    names=["InvVen_actual"],
                ).iloc[:7]
                test = test.reindex(
                    [
                        datetime.strftime(
                            datetime.strptime(date, "%Y/%m/%d"), "%Y-%m-%d"
                        )
                        if "/" in date
                        else date
                        for date in test.index
                    ]
                )
                data[os.path.basename(file).replace(".csv", "")] = pd.concat(
                    [
                        data[os.path.basename(file).replace(".csv", "")],
                        test.loc[
                            data[os.path.basename(file).replace(".csv", "")].index
                        ],
                    ],
                    axis=1,
                )
            except:
                import pdb

                pdb.set_trace()

    # Number of days
    T = 7

    # Penalization term for Number of ventilators to be transported / shared
    λ = 0.5

    # θj = the maximum fraction of ajj used for Uji , that is Uji = θjajj
    θ = 0.2

    # ρj = the maximum fraction of ajj used for Gj , that is Gj = ρjajj
    ρ = 0.6

    # Number of ventilator locations
    N = init_data.shape[0]

    # Adjacency Matrix with 1 if neighbours, 0 if not
    # Rows i ∈ Sj (includes SNS), Columns j ∈ S+ (includes SNS)
    if A is None:
        A = np.array(
            [
                np.sum(
                    [
                        np.array(list(range(51))) == np.array([idx])
                        for idx in eval(str(indices))
                    ],
                    axis=0,
                )
                for _, indices in init_data["nb_idx"].iloc[:-1].iteritems()
            ]
        )
        assert np.alltrue(A == A.T), "Adjacency matrix must be symmetric."
        A = np.concatenate((A, np.zeros((51, 1))), axis=1)
        A = np.concatenate((A, np.zeros((1, 52))), axis=0)
        np.fill_diagonal(A, init_data["Available capacity"].to_numpy())

    J = np.ones((52, 52))
    np.fill_diagonal(J, np.zeros(52))

    K = np.array(
        [
            np.sum(
                [
                    np.array(list(range(51))) == np.array([idx])
                    for idx in eval(str(indices))
                ],
                axis=0,
            )
            for _, indices in init_data["nb_idx"].iloc[:-1].iteritems()
        ]
    )
    assert np.alltrue(K == K.T), "Adjacency matrix must be symmetric."

    # Demand
    def demand(col="InvVen_mean"):
        d_jt = pd.concat(
            [
                init_data["State"]
                .iloc[:-1]
                .apply(
                    lambda state: data[state].loc[date][col]
                    if state in data.keys()
                    else 0
                )
                for date in data["Georgia"].index
            ],
            keys=data["Georgia"].index,
            axis=1,
        )
        d_jt = d_jt.reindex(init_data["State"].iloc[:-1])
        return d_jt

    D = demand(col="InvVen_mean").to_numpy()

    # xij = the number of ventilators shipped
    # from location i ∈ S+ to location j ∈ S+
    # at time 0. The flow matrix is X, with
    # xij as the element at (i,j).
    X = cp.Variable((N, N), name="Number of ventilators shipped", nonneg=True)

    # sjj = the total number of ventilators that
    # are available in j ∈ S+ after initial
    # allocation, and belongs to j.
    # sij = the total number of ventilators that
    # are available in j ∈ S+ after initial
    # allocation, and belongs to i ∈ Sj.
    S = cp.Variable((N, N), name="Total number of available ventilators", nonneg=True)

    # ∆jt = unmet demand for ventilators at
    # location j ∈ S on day t ∈ T . The matrix
    # representation is ∆.
    Δ = cp.Variable((N - 1, T), name="Unmet Demand", nonneg=True)

    # Solve optimization
    prob = cp.Problem(
        cp.Minimize(
            np.ones(N - 1).T @ Δ @ np.ones(T)
            + λ * (cp.quad_form(np.ones((N - 1,)), cp.multiply(K, S[:-1, :-1])))
        ),  # (1a)
        constraints=[
            cp.diag(S[:-1, :-1])
            == cp.diag(A[:-1, :-1])
            - cp.multiply(K, X[:-1, :-1]).T @ np.ones(N - 1)
            + X[-1, :-1],  # (1b)
            S[-1, -1] == A[-1, -1] - X[-1, :-1] @ np.ones(N - 1),  # (1c)
            cp.multiply(K, cp.multiply(J[:-1, :-1], S[:-1, :-1]))
            == cp.multiply(K, cp.multiply(J[:-1, :-1], A[:-1, :-1]))
            + cp.multiply(K, cp.multiply(J[:-1, :-1], X[:-1, :-1])),  # (1d)
            Δ >= 0,
            *[
                Δ[:, t]
                >= D[:, t]
                - cp.diag(S[:-1, :-1])
                - cp.multiply(K, cp.multiply(J[:-1, :-1], S[:-1, :-1])).T
                @ np.ones(N - 1)
                for t in range(T)
            ],  # (1e)
            cp.diag(S) >= ρ * cp.diag(A),  # (1f)
            *[-cp.diag(A) <= X[:, i] for i in range(52)],  # (1g)
            *[X[:, i] <= θ * cp.diag(A) for i in range(52)],  # (1g)
        ],
    )
    prob.solve()

    print(prob.status)
    print(np.ones(N - 1).T @ Δ.value @ np.ones(T))

    A = S.value

optimal_inaccurate
0.0
> <ipython-input-5-e1d7fe4918bb>(32)<module>()
     30     # Test data
     31     root, dirs, files = next(os.walk("./data/project 4 dataset/eval"))
---> 32     for file in files:
     33         if "csv" in file:
     34             try:

ipdb> D.shape
(51, 7)
ipdb> quit


BdbQuit: 

time: 35min 47s


<IPython.core.display.Javascript object>

In [9]:
results = pd.DataFrame(
    [
        [1, 781, 781],
        [2, 4421, 3438],
        [3, 6427, 4078],
        [4, 5949, 4637],
        [5, 3855, 3077],
        [6, 1706, 923],
        [7, 0, 0],
        [8, 0, 0],
        [9, 0, 0],
        [10, 0, 0],
        ["Total", 23139, 16934],
    ],
    columns=["Week", "Point Forecasts", "Forecast Uncertainty - SR"],
)
results.set_index("Week", inplace=True)
results.T

Week,1,2,3,4,5,6,7,8,9,10,Total
Point Forecasts,781,4421,6427,5949,3855,1706,0,0,0,0,23139
Forecast Uncertainty - SR,781,3438,4078,4637,3077,923,0,0,0,0,16934


time: 9.47 ms


<IPython.core.display.Javascript object>

---
## 1) Deterministic Model

Master Problem:

\begin{align}
  \underset{S \geq 0}{\min} \mu & \\
\end{align}

Sub Problem:

\begin{align}
  \underset{F}{\min} h(S=S^* = \underset{S \geq 0}{\arg\min}\,\mu, d=\tilde{d}) &= \underset{F}{\min} \sum^N_{i=1} h_i F_{B_i E_i} + \sum^N_{i=1} \sum^N_{j=1} c_{ij} F_{B_i M_j} + \sum^N_i p_i F_{RM_i} \\
  \text{subject to } &S_i = F_{B_i M_i} + \sum^N_{j =1 \\ i \not= 1} F_{B_i M_j} + F_{B_i E_i}\qquad i = 1, \cdots , N, \\
  &F_{B_i M_i} + \sum^N_{j = 1 \\ j \not= i} F_{B_j M_i} + F_{RM_i} = {d}_i\qquad i = 1, \cdots , N, \\
  &\sum^N_{i=1} {d}_i = \sum^N_{i=1} F_{RM_i} + \sum^N_{i = 1} F_{RE_i}, \\
  &F_{B_i E_i} + F_{RE_i} = S_i\qquad i = 1, \cdots , N, \\
  &F_{B_i E_i}, F_{B_i M_j}, F_{RM_i}, F_{RE_i} \geq 0\qquad i = 1, \cdots , N, \\
  &j = 1, \cdots , N.
\end{align}

In [ ]:
class Benders:
    def __init__(self, c, first_stage_constraints, s_gen, ε=1e-5, log=True):
        """The Bender's method implemented here
        is used to optimize the Two-Stage Stochastic Linear Program
        with second-stage variables following a discrete distribution

        """
        self.c = c  # First-stage decision coefficient
        self.first_stage_constraints = first_stage_constraints  # Function that gives a list of the constraints for first-stage problem
        self.s_gen = s_gen  # Function to generate scenario specific entities
        self.ε = ε  # Tolerance between Upper Bound and Lower Bound
        self.log = log  # Whether or not to log progress

    def solve(self):
        """Solves the Two-stage Stochastic Optimization"""
        # 1. Initialize UB = infinity, LB = -infinity
        UB, LB = np.inf, -np.inf
        x = cp.Variable(self.c.shape, nonneg=True)  # First-stage decision variables

        # Master Problem
        z = cp.Variable(1, nonneg=True)
        MP_obj = cp.Minimize(z)

        # Bender cuts that we will be adding to the master problem
        cuts = (
            self.first_stage_constraints
            if self.first_stage_constraints is not None
            else []
        )

        k = 0

        # 2. While UB - LB > ε, we will continue adding cuts
        while UB - LB > self.ε:

            expected_h = 0
            α, β = 0, 0  # Hyperplane of cut coefficients

            # Go through all the scenarios possible
            for p_s, d_s, C_s, D_s, ξ_s in self.s_gen():

                # 3. Solve the Dual of the Sub-problem for all combinations of demand \tilde{d}
                # [Equivalent to solving primal since LPs have 0 duality gap]
                π_optimal, h_dual_optimal, status = self.second_stage(
                    x=x, d=d_s, C=C_s, D=D_s, ξ=ξ_s
                )

                #                 # 4a. If dual variables are unbounded, add cut
                #                 if status == "infeasible":
                #                     cuts += [(ξ_s - C_s @ x).T @ π_optimal <= 0]

                #                 # 4b. Else set UB and add cut on first stage primal optimal value
                #                 else:
                #                     expected_h += p_s * h_dual_optimal
                #                     cuts += [z >= (ξ_s - C_s @ x).T @ π_optimal]

                expected_h += p_s * h_dual_optimal
                α += p_s * ξ_s.T @ π_optimal
                β += -p_s * C_s.T @ π_optimal

            cuts += [z >= α + β.T @ x]

            # Update UB
            UB = min(
                UB,
                self.c.T @ x.value + expected_h if x.value is not None else expected_h,
            )

            # 5. Solve Master Problem
            MP_prob = cp.Problem(MP_obj, constraints=cuts)
            MP_prob.solve()

            # 6. Update LB
            LB = MP_prob.value

            if self.log:
                print("=" * 20 + f" Iteration {k}" + "=" * 20)
                print(
                    f"Lower Bound:",
                    np.round(LB, 3),
                    "|",
                    f"Upper Bound:",
                    np.round(UB, 3),
                )
            k += 1

        if self.log:
            print(f"Optimal primal objective of first stage problem: {np.round(LB, 3)}")
            print(f"Optimal first-stage decision variables: {np.round(x.value, 3)}")

        return LB, np.array(x.value)

    def second_stage(self, x, d, C, D, ξ):
        """Dual Form of Second-stage Optimization"""

        π = cp.Variable(
            ξ.shape, nonneg=False
        )  # Second-stage dual variables are Free since they are dual of the equality constraints
        constraints = [D.T @ π <= d]  # Second-stage dual constraints
        obj = cp.Maximize(
            (ξ - C @ (np.zeros(C.shape[1]) if x.value is None else x.value)).T @ π
        )
        prob = cp.Problem(obj, constraints=constraints)
        prob.solve()

        return π.value, prob.value, prob.status

In [ ]:
def get_transshipment_variables():

    # Number of retailers
    N = 4

    # Holding cost Vector
    h = np.array([1] * N)

    # Shortage cost Vector
    p = np.array([4] * N)

    # Cost of transshipment from retailer i to retailer j Matrix
    C = np.full((N, N), 0.5)
    np.fill_diagonal(C, 0)

    # PMF of Retailer Demand distribution
    P = np.array([0.020, 0.14, 0.68, 0.14, 0.020])

    # Each Retailer's Demand distribution associated with probabilities above
    RETAILER_DEMANDS = np.array(
        [
            [60, 80, 100, 120, 140],
            [100, 150, 200, 250, 300],
            [90, 120, 150, 180, 210],
            [70, 120, 170, 220, 270],
        ]
    )

    return N, h, p, C, P, RETAILER_DEMANDS


def get_second_stage_variables(s):
    """"""

    N, h, p, C, P, RETAILER_DEMANDS = get_transshipment_variables()

    # Coefficient of Second-stage decision variables
    d_s = np.concatenate(
        (
            h,  # Coefficient of Amount of inventory that moves from begining to ending (FBE)
            C.flatten(),  # Coefficient of Amount of inventory to ship from current retailer to other retailers (FBM)
            p,  # Coefficient of Amount to ship from supplier to retailer's middle inventory (FRM)
            np.array(
                [0] * N
            ),  # Coefficient of Amount to ship from supplier to retailer's ending inventory (FRE)
        )
    )

    # Second-stage C(s) matrix
    C_s = np.concatenate(
        (
            np.eye(N),
            np.zeros(shape=(N, N)),
            np.zeros(shape=(1, N)),
            np.eye(N),
        ),
        axis=0,
    )

    # Second-stage D(s) matrix
    D_s = np.concatenate(
        (
            np.concatenate(
                (
                    -np.eye(N),
                    np.array(
                        [
                            ([-1] * N + [0] * (N * (N - 1)))[-N * idx :]
                            + ([-1] * N + [0] * (N * (N - 1)))[: -N * idx]
                            for idx in range(N)
                        ]
                    ),
                    np.zeros(shape=(N, N)),
                    np.zeros(shape=(N, N)),
                ),
                axis=1,
            ),
            np.concatenate(
                (
                    np.zeros(shape=(N, N)),
                    np.array(
                        [
                            np.array(
                                [
                                    ([1] + [0] * (N - 1))[-idx:]
                                    + ([1] + [0] * (N - 1))[:-idx]
                                ]
                                * N
                            ).flatten()
                            for idx in range(N)
                        ]
                    ),
                    np.eye(N),
                    np.zeros(shape=(N, N)),
                ),
                axis=1,
            ),
            np.array([[0] * N + [0] * (N * N) + [1] * N + [1] * N]),
            np.concatenate(
                (
                    -np.eye(N),
                    np.zeros(shape=(N, N * N)),
                    np.zeros(shape=(N, N)),
                    -np.eye(N),
                ),
                axis=1,
            ),
        ),
        axis=0,
    )

    # ξ(s) vector
    ξ_s = np.concatenate(
        (
            np.zeros(N),
            s,
            np.array([np.sum(s)]),
            np.zeros(N),
        ),
        axis=0,
    )

    return d_s, C_s, D_s, ξ_s


def s_gen_sgd(n=50):
    """Function that generates the scenario dependent
    second-stage variables by sampling demand

    """

    N, h, p, C, P, RETAILER_DEMANDS = get_transshipment_variables()

    for _ in range(n):

        # Get Second-stage variables
        s, p_s = random_demand(P=P, retailer_demands=RETAILER_DEMANDS)

        # Get scenario-dependent variables
        d_s, C_s, D_s, ξ_s = get_second_stage_variables(s)

        yield p_s, d_s, C_s, D_s, ξ_s


def s_gen_benders():
    """Function that generates the scenario dependent
    second-stage variables by enumerating all combinations of demand

    """

    N, h, p, C, P, RETAILER_DEMANDS = get_transshipment_variables()

    # Get Second-stage variables
    for s in itertools.product(*RETAILER_DEMANDS):

        p_s = np.prod(
            [
                P[np.argwhere(retailer_demand == D_i)[0][0]]
                for retailer_demand, D_i in zip(RETAILER_DEMANDS, s)
            ]
        )

        # Get scenario-dependent variables
        d_s, C_s, D_s, ξ_s = get_second_stage_variables(s)

        yield p_s, d_s, C_s, D_s, ξ_s

In [ ]:
benders = Benders(
    c=np.array([0] * 4),
    first_stage_constraints=None,
    s_gen=s_gen_benders,
)

benders.solve()